In [11]:
import copy
import csv
import cvxpy
import datetime
import json
import math
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Pool
import numpy as np
import os
import pandas as pd
import pickle
import random
import seaborn as sns
from typing import List, Dict
import sys
sys.path.append('../')

from tqdm import tqdm

from skyburst import Job
from skyburst.plot import *
from skyburst.traces import philly
from skyburst.traces import philly_utils

import logging
logger = logging.getLogger()

logging.basicConfig(stream=sys.stdout, level=logging.INFO)

In [12]:
jobs = philly.load_philly_traces('~/philly-traces/trace-data')
jobs.sort(key=lambda j: j._submitted_time)
all_jobs = jobs

In [13]:
from math import floor, ceil

# Maximum job time in hours
CLIP_MAX_TIME = 1e9

# Deadline Multiplier d_t = a_t + K * r_t
DEADLINE_CONSTANT = 1.25

# Returns the deadline of a job.
# For continuous simulator
def deadline_mult_fn(arrival_time, run_time, deadline_mult=DEADLINE_CONSTANT):
    waiting_time = (deadline_mult - 1)*run_time
    waiting_time = max(1/12.0, waiting_time)
    return arrival_time + waiting_time + run_time

def deadline_ljw(arrival_time, run_time, deadline_mult=DEADLINE_CONSTANT, run_thres=8.0):
    if run_time < run_thres:
        return arrival_time + run_time
    waiting_time = (deadline_mult - 1)*run_time
    return arrival_time + waiting_time + run_time

def deadline_infinite_fn(arrival_time, run_time):
    return 1e12

def deadline_zero_fn(arrival_time, run_time):
    return arrival_time + run_time

def deadline_constant_fn(arrival_time, run_time, wait_time=1):
    return arrival_time + wait_time + run_time

# Returns the total cost of a job.
def gpu_cost_fn(resources: dict, run_time: float):
    return resources['GPUs'] * run_time

def time_thres_fn(job, run_thres):
    return job.runtime >= run_thres

# NUM_GPUS per node: 8
GPUS_PER_NODE = 8

# Num nodes
NUM_NODES = 16

def preprocess_jobs(jobs: List['JobTrace'], deadline_fn= deadline_infinite_fn , cost_fn=gpu_cost_fn, clip_time: float = CLIP_MAX_TIME):
    """Converts a list of jobs into (arrival_time, run_time, deadline, resources, cost).
    """
    jobs = jobs.copy()
    # Remove invalid jobs (jobs that have not finished and jobs that failed/killed early)
    jobs = [j for j in jobs if j._run_time is not None and j.status=='Pass']
    jobs.sort(key=lambda j: j._submitted_time)
    
    if not clip_time:
        clip_time = 1e9
    
    # Arrival time for jobs
    start_time = jobs[0]._submitted_time
    arrival_times = [(j._submitted_time- start_time).total_seconds()/3600.0 for j in jobs]
    
    # Run time for jobs
    run_times = []
    for j in jobs:
        # Clip the maximum run time for jobs to simplify simulation.
        run_time_hr = j._run_time/60.0
        if run_time_hr > clip_time:
            run_times.append(clip_time)
        else:
            run_times.append(run_time_hr)
    
    deadlines = [deadline_fn(arrival, run) for arrival, run in zip(arrival_times, run_times)]
    
    # Get GPU resources
    resources = []
    for j in jobs:
        detail_dict = j.attempts[-1]['detail']
        gpu_count = sum([len(node_dict['gpus']) for node_dict in detail_dict])
        resources.append({'GPUs': gpu_count})
    
    costs = [cost_fn(res, run) for res, run in zip(resources, run_times)]
    
    return [Job(idx, arr, run, dead, res, cost) \
            for idx, (arr, run, dead, res, cost) in \
            enumerate(list(zip(arrival_times, run_times, deadlines, resources, costs)))]

def generate_philly_jobs(jobs: List['JobTrace'], num_jobs = 360000, deadline_fn= deadline_infinite_fn , cost_fn=gpu_cost_fn, clip_time: float = CLIP_MAX_TIME):
    """Converts a list of jobs into (arrival_time, run_time, deadline, resources, cost).
    """
    num_jobs = int(num_jobs)
    jobs = jobs.copy()
    # Remove invalid jobs (jobs that have not finished and jobs that failed/killed early)
    jobs = [j for j in jobs if j._run_time is not None and j.status=='Pass']
    jobs.sort(key=lambda j: j._submitted_time)
    
    if not clip_time:
        clip_time = 1e9
    
    # Arrival time for jobs
    start_time = jobs[0]._submitted_time
    arrival_times = [(j._submitted_time- start_time).total_seconds()/3600.0 for j in jobs]

    interarrival_times = [arrival_times[i+1]-arrival_times[i] for i in range(len(arrival_times)-1)]
    
    # Run time for jobs
    run_times = []
    for j in jobs:
        # Clip the maximum run time for jobs to simplify simulation.
        run_time_hr = j._run_time/60.0
        if run_time_hr > clip_time:
            run_times.append(clip_time)
        else:
            run_times.append(run_time_hr)

    # Get GPU resources
    resources = []
    for j in jobs:
        detail_dict = j.attempts[-1]['detail']
        gpu_count = sum([len(node_dict['gpus']) for node_dict in detail_dict])
        resources.append({'GPUs': gpu_count})
    
    import random
    sampled_arrival_times = np.random.choice(interarrival_times, size=num_jobs, replace=True)
    job_indexes = np.random.choice(list(range(len(run_times))), size=num_jobs, replace=True)
    proc_jobs = []
    arrival_time = 0 
    for idx in range(num_jobs):
        job_idx = job_indexes[idx]
        runtime = run_times[job_idx]
        deadline = deadline_fn(arrival_time, runtime)
        cost = cost_fn(resources[job_idx], runtime)
        proc_jobs.append(Job(idx, arrival_time, runtime, deadline, resources[job_idx], cost))
        arrival_time += sampled_arrival_times[idx]
    
    return proc_jobs

In [14]:
class Node(object):
    def __init__(self, num_gpus):
        self.num_gpus = num_gpus
        self.gpu_dict = {}
        for idx in range(self.num_gpus):
            self.gpu_dict[idx] = None
    
    def free_gpus(self):
        return len([v for v in self.gpu_dict.values() if not isinstance(v, Job)])
    
    def try_fit(self, cur_timestamp, job, num_gpus=None):
        gpu_list = []
        if num_gpus==None:
            num_gpus = job.resources['GPUs']

        if num_gpus > self.free_gpus():
            return False, gpu_list

        counter =0
        for idx in range(self.num_gpus):
            gpu_val = self.gpu_dict[idx]
            if not isinstance(gpu_val, Job):
                self.gpu_dict[idx] = job
                gpu_list.append(idx)
                counter +=1
            if counter >= num_gpus:
                break
        return True, gpu_list
    
    def __repr__(self):
        return f'{self.gpu_dict}'        
        

class Cluster(object):
    
    def __init__(self, num_nodes, num_gpus_per_node, preempt=None, reserve=False):
        self.num_nodes = num_nodes
        self.num_gpus_per_node = num_gpus_per_node
        self.nodes = [Node(num_gpus_per_node) for _ in range(num_nodes)]
        self.preempt = preempt
        self.reserve = reserve
        self.active_jobs = {}
    
    def is_full(self):
        for n in self.nodes:
            for gpu_idx in n.gpu_dict:
                if not n.gpu_dict[gpu_idx]:
                    return False
        return True
    
    def try_fit_v2(self, cur_timestamp, job):
        num_gpus = job.resources['GPUs']
                        
        # Generate Plan
        free_gpus = [n.free_gpus() for n in self.nodes]
        free_plus_preempted_gpus = free_gpus.copy()
        node_idx_to_pre_job = {}
        
        # Logic to pre-empt GPU(s). Generates candidates jobs to be pre-empted on the cluster.
        preemptible_jobs = []
        if self.preempt:
            C = 1.0/self.preempt - 1.0
            for j_idx, j in self.active_jobs.items():
                # Filter for jobs that will not exceed deadline (if preempted)
                if j.deadline - j.runtime > cur_timestamp and job.runtime > j.runtime:
                    # job.cost - j.cost >= j.opp_cost:
                    factor = 0
                    if j.num_gpus > job.num_gpus:
                        factor = (1 - (job.num_gpus/j.num_gpus))*job.runtime
                    if j.num_gpus ==0:
                        continue
                    if job.runtime - j.runtime >= C * (cur_timestamp - j.start) + factor + j.opp_cost/(self.preempt*j.num_gpus):
                        preemptible_jobs.append(j)                  
            preemptible_jobs.sort(key=lambda x: x.cost)
        
        # No hope of fitting onto cluster :(
        if num_gpus > sum(free_plus_preempted_gpus):
            return False, []
        
        # Generate job GPU demands
        if num_gpus > self.num_gpus_per_node:
            # Multinode case, i.e. 26 GPUs, 8 GPU/node cluster -> job_gpu_demands = [8,8,8,2]
            job_gpu_demands = [self.num_gpus_per_node]*int(num_gpus/self.num_gpus_per_node)
            if num_gpus%self.num_gpus_per_node:
                job_gpu_demands.append(num_gpus%self.num_gpus_per_node)
        else:
            job_gpu_demands = [num_gpus]

        # Generate Plans - which nodes to place job on and which jobs to preempt
        # Maps node idx to gpu_demand
        node_idx_taken = {}
        # Maps node idx to (gpu_demand, list of job idx of preempted jobs on that node)
        preempted_idx_taken = {}
        remove_idx = []
        
        # Go through free space only first, generate partial plan with free space
        for list_idx, gpu_demand in enumerate(list(job_gpu_demands)):
            for n_idx, n in enumerate(self.nodes):
                if n_idx in node_idx_taken:
                    continue
                if free_gpus[n_idx] >= gpu_demand:
                    node_idx_taken[n_idx] = gpu_demand
                    job_gpu_demands.remove(gpu_demand)
                    break

        # Go through premptible jobs to find more space, generate final plan
        if self.preempt and job_gpu_demands:
            temp_free_gpus = free_gpus.copy()
            preemptible_jobs_to_node = [[] for _ in range(self.num_nodes)]
            for pre_j in preemptible_jobs:
                for n_idx, n_gpu_list in pre_j.allocated_gpus.items():
                    temp_free_gpus[n_idx] += len(n_gpu_list)
                    preemptible_jobs_to_node[n_idx].append(pre_j)
                
                for list_idx, gpu_demand in enumerate(list(job_gpu_demands)):
                    for n_idx, n in enumerate(self.nodes):
                        if n_idx in node_idx_taken or n_idx in preempted_idx_taken:
                            continue
                        if temp_free_gpus[n_idx] >= gpu_demand:
                            preempted_idx_taken[n_idx] = (gpu_demand, [j.idx for j in preemptible_jobs_to_node[n_idx]]) 
                            job_gpu_demands.remove(gpu_demand)
                            break
                
                if not job_gpu_demands:
                    break
            
        # If there are still demands that cannot be satisifed via free and preempted jobs,
        # it cannot be scheduled on the cluster.
        if job_gpu_demands:
            return False, []
        
        # Execute plan
        preempted_jobs = []
        # Populate nodes that enough free gpus
        for n_idx, gpu_demand in node_idx_taken.items():
            gpu_list = self.try_fit_node(self.nodes[n_idx], cur_timestamp, job, num_gpus=gpu_demand)
            job.allocated_gpus[n_idx] = gpu_list
        
        if self.preempt:
            # Poppulate nodes that need jobs preempted first
            buffer = []
            for n_idx, job_tuple in preempted_idx_taken.items():
                gpu_demand, preempted_job_idxs = job_tuple
                for job_idx in preempted_job_idxs:
                    if job_idx in buffer:
                        continue
                    pre_job = self.active_jobs[job_idx]
                    job.opp_cost += (cur_timestamp - pre_job.start) * pre_job.num_gpus
                    preempted_job = self.preempt_job(job_idx)
                    preempted_jobs.append(preempted_job)
                    buffer.append(job_idx)
                gpu_list = self.try_fit_node(self.nodes[n_idx], cur_timestamp, job, num_gpus=gpu_demand)
                job.allocated_gpus[n_idx] = gpu_list
        
        job.start = cur_timestamp
        self.active_jobs[job.idx] = job
        
        return True, preempted_jobs
    
    def try_fit_node(self, node, cur_timestamp, job, num_gpus=None):
        gpu_list = []
        if num_gpus==None:
            num_gpus = job.resources['GPUs']

        if num_gpus > node.free_gpus():
            raise ValueError("Should not go here in try_fit_node!")

        counter =0
        for idx in range(node.num_gpus):
            gpu_val = node.gpu_dict[idx]
            if not isinstance(gpu_val, Job):
                node.gpu_dict[idx] = job
                gpu_list.append(idx)
                counter +=1
            if counter >= num_gpus:
                break
        return gpu_list
    
    def preempt_job(self, job_idx):
        assert job_idx in self.active_jobs
        r_job = self.active_jobs[job_idx]
        for n_idx, gpu_list in r_job.allocated_gpus.items():
            node = self.nodes[n_idx]
            for gpu_idx in gpu_list:
                node.gpu_dict[gpu_idx] = None
        r_job.start = None
        r_job.allocated_gpus = {}
        del self.active_jobs[job_idx]
        return r_job
            
    def try_clear(self, t: float):
        """Clears cluster of completed jobs.
        """
        completed_jobs = []
        for job_idx, job in self.active_jobs.items():
            if t >= job.start + job.runtime:
                node_gpu_dict = job.allocated_gpus
                for node_idx, gpu_list in node_gpu_dict.items():
                    node_gpu_dict = self.nodes[node_idx].gpu_dict
                    for gpu_idx in gpu_list:
                        node_gpu_dict[gpu_idx] = None
                completed_jobs.append(job)
        
        for job in completed_jobs:
            job.state = 'LOCAL'
            del self.active_jobs[job.idx]
        return completed_jobs
    
    def __repr__(self):
        repr_str = 'Cluster State:\n'
        for idx, n in enumerate(self.nodes):
            repr_str += f'Node {idx}: {n}\n'
        return repr_str



In [21]:
## Attempt Pre-emption
# Add job into the cluster -> pre empt jobs (what jobs to preempt?)
# Only preempt jobs that have reached deadline & are cheaper than the current job
# Preempted jobs go back into the queue
# Warning very slow!
def continuous_simulator(jobs: List[Job], policy, num_nodes, job_filter_policy=None, preempt=None, reserve=False, verbose=False, debug=False):
    if '-' in policy:
        policy, mod = policy.split('-')
    else:
        mod = None
    
    if policy == 'fifo':
        sort_func = lambda x: x.arrival
    elif policy == 'lifo':
        sort_func = lambda x: -x.arrival
    elif policy == 'edf':
        sort_func = lambda x: x.deadline
    elif policy == 'ldf':
        sort_func = lambda x: -x.deadline
    elif policy == 'sjf':
        sort_func = lambda x: x.runtime
    elif policy == 'svjf':
        sort_func = lambda x: x.cost
    elif policy == 'ljf':
        sort_func = lambda x: -x.runtime
    elif policy == 'lvjf':
        sort_func = lambda x: -x.cost
    elif policy == 'swf':
        sort_func = lambda x: x.deadline - x.runtime 
    elif policy == 'lwf':
        sort_func = lambda x: -x.deadline + x.runtime
    else:
        pass
        #raise ValueError(f'Policy {policy} does not match!')
    
    remove_hol = False
    if mod == 'nohol':
        remove_hol = True
    
    job_filter_fn = None
    if job_filter_policy is not None:
        job_filter_policy, run_thres = job_filter_policy.split('-')
        if job_filter_policy == 'ljw':
            job_filter_fn = lambda x: time_thres_fn(x, float(run_thres))
        else:
            raise ValueError(f'Incorrect job filter policy {job_filter_policy}.')
    finished_jobs = []
    jobs = copy.deepcopy(jobs)
    num_jobs = len(jobs)

    queue = []
    cloud_cost = 0
    cluster = Cluster(num_nodes=num_nodes, num_gpus_per_node=GPUS_PER_NODE, preempt=preempt, reserve=reserve)
    t=0
    #tqdm_iter = tqdm(total=len(jobs), desc ="# of Jobs left")
    while len(jobs) >0 or len(queue)>0:
        # Clear cluster of jobs that have completed
        finished_jobs.extend(cluster.try_clear(t))
        
        # Check for jobs that have waited too long (move to cloud).
        for job in queue[:]:
            if t >= job.deadline - job.runtime:
                queue.remove(job)
                job.state = 'TIMEOUT-CLOUD'
                finished_jobs.append(job)
        
        # Add jobs to queue that have arrived
        for job in jobs[:]:
            if job.arrival < t:
                raise ValueError("Should not have entered here!")
                
            if job.arrival == t:
                #print(job.idx)
                jobs.remove(job)
                #tqdm_iter.update(1)
                if job.idx%1000==0:
                    logging.info(f'{policy}-{num_nodes}: {job.idx}')
                if job_filter_fn is not None:
                    # If arrival job does not satisfy job filter, move job to cloud.
                    if not job_filter_fn(job):
                        job.state = 'FILTER-CLOUD'
                        finished_jobs.append(job)
                        continue
                queue.append(job)
            else:
                break
        queue.sort(key = sort_func)
        
        # Go through queue and fit jobs onto cluster as needed
        preempted_jobs = []
        for job in queue[:]:
            can_fit, preempted_jobs = cluster.try_fit_v2(t, job)
            if can_fit:
                queue.remove(job)
                queue.extend(preempted_jobs)
            elif not remove_hol:
                break
        
        next_time_list = []
        
        # Skip to next timestep where a new job either arrives or when a job finishes on the cluster.
        for job_idx, job in cluster.active_jobs.items():
            release_time = job.start + job.runtime
            next_time_list.append(release_time)

        if len(jobs)>0:
            next_time_list.append(jobs[0].arrival)
        
        if len(next_time_list)==0:
            assert len(queue)==0 and len(jobs) ==0
            break
        
        if min(next_time_list) < t:
            raise ValueError('Simulator is severely bugged; Raise a Git Issue.')
                
        if verbose or debug:
            print(f'Timestamp: {t}')
            print(f'Jobs left: {len(jobs)}')
            print(f'Cloud Cost: {cloud_cost}')
            print(cluster)
            print('Queue: ' + str(queue) + '\n')
            print('Finished Jobs: ' + str(finished_jobs) + '\n')
            if debug:
                import pdb; pdb.set_trace()
        
        t = min(next_time_list)
    finished_jobs.extend(cluster.try_clear(1e12))
    assert len(jobs)==0 and len(queue)==0

    # Compute Avg. Waiting Time
    total_waiting_time = 0.0
    num_jobs = 0
    for job in finished_jobs:
        # remove warmup and cooldown period
        if job.idx < 5000 or job.idx > len(finished_jobs)-5000:
            continue
        # Moved to cloud
        if job.start is None:
            if job.state == 'TIMEOUT-CLOUD':
                total_waiting_time += job.deadline - job.runtime - job.arrival
            elif job.state == 'FILTER-CLOUD':
                pass
            else:
                raise ValueError('Invalid State!')
            cloud_cost += job.cost
        else:
            assert job.state == 'LOCAL'
            total_waiting_time += job.start - job.arrival
        num_jobs+=1
    

    print(f'{policy}-{num_nodes}')
    print(f'Cloud Cost ({policy}-{num_nodes}): {cloud_cost}')
    print(f'Avg Waiting Time ({policy}-{num_nodes}): {total_waiting_time/num_jobs}')
    print(len(finished_jobs))
    return f'{policy}-{num_nodes}', save_simulator_results(finished_jobs)

def save_simulator_results(finished_jobs: List[Job]) -> dict:
    result_dict = {
        'idx': [],
        'arrival': [],
        'start': [],
        'runtime': [],
        'deadline': [],
        'num_gpus': [],
        'state': [],
        'allocated_gpus': [],
    }
    finished_jobs.sort(key = lambda x: x.idx)
    for job in finished_jobs:
        result_dict['idx'].append(job.idx)
        result_dict['arrival'].append(job.arrival)
        if job.start is not None:
            result_dict['start'].append(job.start)
        else:
            result_dict['start'].append(-1)
        result_dict['runtime'].append(job.runtime)
        result_dict['deadline'].append(job.deadline)
        result_dict['num_gpus'].append(job.num_gpus)
        result_dict['state'].append(job.state)
        result_dict['allocated_gpus'].append(job.allocated_gpus)
    result_dict['idx'] = np.array(result_dict['idx'])
    result_dict['arrival'] = np.array(result_dict['arrival'])
    result_dict['start'] = np.array(result_dict['start'])
    result_dict['runtime'] = np.array(result_dict['runtime'])
    result_dict['deadline'] = np.array(result_dict['deadline'])
    result_dict['num_gpus'] = np.array(result_dict['num_gpus'])
    result_dict['state'] = np.array(result_dict['state'])
    result_dict['allocated_gpus'] = np.array(result_dict['allocated_gpus'])
    return result_dict

In [23]:
# def continuous_hyperparameter_search(jobs, algs, nodes_list, deadline_fn, job_filter_policy=None, preempt=None, remove_hol=False):
#     jobs_proc = generate_philly_jobs(jobs, num_jobs=200000, deadline_fn=deadline_fn)
#     tuples = []
#     for n_nodes in nodes_list:
#         for alg in algs:
#             tuples.append((jobs_proc.copy(), alg, n_nodes, job_filter_policy, preempt, remove_hol))
#     results = []
#     with multiprocessing.Pool(processes=32) as pool:
#         results = pool.starmap(continuous_simulator, tuples)
#     results_dict = {}
#     for r in results:
#         results_dict[r[0]] = r[1]
#     return results_dict


# ajw_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, deadline_infinite_fn)
# file = open('logs/ajw_philly_cont.log', 'wb')
# pickle.dump(ajw_results, file)
# file.close()

# njw_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, deadline_zero_fn)
# file = open('logs/njw_philly_cont.log', 'wb')
# pickle.dump(njw_results, file)
# file.close()

# num_nodes = list(range(24, 150, 4))
# algs = ['fifo', 'lifo', 'swf']
# for ddl in [1.25]:
#     if ddl==1:
#         temp_fn = deadline_zero_fn
#     elif ddl == 100000:
#         temp_fn = deadline_infinite_fn
#     else:
#         temp_fn = lambda x, y: deadline_mult_fn(x,y,deadline_mult=ddl)
#     hybrid_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, \
#                                                       deadline_fn=temp_fn)
#     file = open(f'logs/generated/vjw_philly_{ddl}_cont.log', 'wb')
#     pickle.dump(hybrid_results, file)
#     file.close()

temp_fn = lambda x, y: deadline_mult_fn(x,y,deadline_mult=1.25)
jobs_proc = preprocess_jobs(all_jobs, deadline_fn=temp_fn)

print(len(jobs_proc))
asdf = [j.deadline - j.runtime - j.arrival for j in jobs_proc]
print(sum(asdf)/len(asdf))

# jobs_proc = generate_philly_jobs(all_jobs, num_jobs=320000, deadline_fn=temp_fn)
asdf = continuous_simulator(jobs_proc, 'swf', 72)
asdf1 = continuous_simulator(jobs_proc, 'fifo', 72)
# asdf2 = continuous_simulator(jobs_proc, 'lifo', 72)
#asdf1 = continuous_simulator(jobs_proc, 'swf-nohol', 72)
# asdf1 = continuous_simulator(jobs_proc, 'sjf', 72)

# #asdf1 = continuous_simulator(jobs_proc, 'fifo', 72, asdf=True)#job_filter_policy='ljw-1')

83154
1.0456151822053108


# of Jobs left: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83154/83154 [00:40<00:00, 2028.25it/s]

swf-72
Cloud Cost (swf-72): 122426.03722222186
Avg Waiting Time (swf-72): 0.4153995132101629
83154



# of Jobs left: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 83154/83154 [00:29<00:00, 2820.90it/s]

fifo-72
Cloud Cost (fifo-72): 138690.16638888972
Avg Waiting Time (fifo-72): 0.37448066225436977
83154


In [17]:
algs = ['fifo', 'lifo', 'sjf', 'svjf', 'ljf', 'lvjf', 'edf', 'ldf', 'swf', 'lwf']

#num_nodes = [72, 76, 80, 84, 88, 92, 96, 100, 104, 108, 112, 116, 120, 124, 128, 132, 136] #[105, 110, 120, 125]
num_nodes = list(range(24, 150, 4))

def continuous_hyperparameter_search(jobs, algs, nodes_list, deadline_fn, job_filter_policy=None, preempt=None, remove_hol=False):
    jobs_proc = preprocess_jobs(jobs, deadline_fn=deadline_fn)
    tuples = []
    for n_nodes in nodes_list:
        for alg in algs:
            tuples.append((jobs_proc, alg, n_nodes, job_filter_policy, preempt, remove_hol))
    results = []
    with multiprocessing.Pool(processes=63) as pool:
        results = pool.starmap(continuous_simulator, tuples)
    results_dict = {}
    for r in results:
        results_dict[r[0]] = r[1]
    return results_dict


# ajw_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, deadline_infinite_fn)
# file = open('logs/ajw_philly_cont.log', 'wb')
# pickle.dump(ajw_results, file)
# file.close()

# njw_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, deadline_zero_fn)
# file = open('logs/njw_philly_cont.log', 'wb')
# pickle.dump(njw_results, file)
# file.close()

print(algs)
for hours_wait in [0, 0.5, 1, 2]:
    temp_fn = lambda x,y: deadline_constant_fn(x, y, wait_time=hours_wait)
    hybrid_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, \
                                                      deadline_fn=temp_fn)
    file = open(f'logs/constant/cjw_philly_{hours_wait}_cont.log', 'wb')
    pickle.dump(hybrid_results, file)
    file.close()


# for ddl in [1, 1.1, 1.25, 1.5, 2, 100000]:
#     if ddl==1:
#         temp_fn = deadline_zero_fn
#     elif ddl == 100000:
#         temp_fn = deadline_infinite_fn
#     else:
#         temp_fn = lambda x, y: deadline_mult_fn(x,y,deadline_mult=ddl)
#     hybrid_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, \
#                                                       deadline_fn=temp_fn)
#     file = open(f'logs/base/vjw_philly_{ddl}_cont.log', 'wb')
#     pickle.dump(hybrid_results, file)
#     file.close()

# for ddl in [1.5]:
#     for preempt_thres in [1/10.0, 1/8.0, 1/5.0, 1/2.0]:
#         temp_fn = lambda x, y: deadline_mult_fn(x,y,deadline_mult=ddl)
#         hybrid_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, \
#                                                           deadline_fn=temp_fn, preempt=preempt_thres)
#         file = open(f'logs/vjw_philly_{ddl}_preempt_{preempt_thres}_cont.log', 'wb')
#         pickle.dump(hybrid_results, file)
#         file.close()

# def deadline_ljw(arrival_time, run_time, deadline_mult=DEADLINE_CONSTANT, run_thres=8.0):
#     if run_time < run_thres:
#         return arrival_time + run_time
#     waiting_time = (deadline_mult - 1)*run_time
#     return arrival_time + waiting_time + run_time
# for run_thres in [1]:
#     for ddl in [1.5]:
#         temp_fn = lambda x, y: deadline_mult_fn(x,y,deadline_mult=ddl)
#         hybrid_results = continuous_hyperparameter_search(all_jobs, algs, num_nodes, \
#                                                           deadline_fn=temp_fn, job_filter_policy = f'ljw-{run_thres}')
#         file = open(f'logs/vjw_philly_{ddl}_{run_thres}_cont.log', 'wb')
#         pickle.dump(hybrid_results, file)
#         file.close()



['fifo', 'lifo', 'sjf', 'svjf', 'ljf', 'lvjf', 'edf', 'ldf', 'swf', 'lwf']
INFO:root:fifo-24: 0
INFO:root:sjf-24: 0
INFO:root:ljf-24: 0
INFO:root:fifo-24: 1000
INFO:root:sjf-24: 1000
INFO:root:edf-24: 0
INFO:root:swf-24: 0
INFO:root:ljf-24: 1000
INFO:root:fifo-24: 2000
INFO:root:fifo-28: 0
INFO:root:sjf-24: 2000
INFO:root:edf-24: 1000
INFO:root:sjf-28: 0
INFO:root:swf-24: 1000
INFO:root:ljf-28: 0
INFO:root:ljf-24: 2000
INFO:root:fifo-28: 1000
INFO:root:fifo-24: 3000
INFO:root:edf-24: 2000
INFO:root:edf-28: 0
INFO:root:swf-24: 2000
INFO:root:sjf-24: 3000
INFO:root:sjf-28: 1000
INFO:root:swf-28: 0
INFO:root:ljf-28: 1000
INFO:root:fifo-28: 2000
INFO:root:fifo-32: 0
INFO:root:ljf-24: 3000
INFO:root:fifo-24: 4000
INFO:root:edf-24: 3000
INFO:root:sjf-32: 0
INFO:root:edf-28: 1000
INFO:root:swf-24: 3000
INFO:root:sjf-24: 4000
INFO:root:ljf-28: 2000
INFO:root:sjf-28: 2000
INFO:root:fifo-28: 3000
INFO:root:ljf-32: 0
INFO:root:edf-24: 4000
INFO:root:edf-32: 0
INFO:root:swf-28: 1000
INFO:root:edf-

INFO:root:sjf-24: 12000
INFO:root:sjf-56: 3000
INFO:root:edf-52: 3000
INFO:root:sjf-40: 5000
INFO:root:sjf-52: 3000
INFO:root:swf-60: 2000
INFO:root:swf-68: 0
INFO:root:ljf-40: 6000
INFO:root:ljf-32: 7000
INFO:root:edf-32: 10000
INFO:root:ljf-52: 3000
INFO:root:edf-48: 4000
INFO:root:sjf-60: 2000
INFO:root:fifo-72: 0
INFO:root:swf-36: 8000
INFO:root:fifo-32: 7000
INFO:root:ljf-28: 13000
INFO:root:edf-44: 7000
INFO:root:sjf-44: 5000
INFO:root:swf-56: 3000
INFO:root:ljf-24: 16000
INFO:root:ljf-56: 3000
INFO:root:fifo-64: 1000
INFO:root:sjf-32: 7000
INFO:root:fifo-28: 13000
INFO:root:sjf-72: 0
INFO:root:ljf-60: 2000
INFO:root:fifo-36: 8000
INFO:root:edf-40: 5000
INFO:root:fifo-24: 13000
INFO:root:fifo-56: 3000
INFO:root:edf-28: 10000
INFO:root:sjf-64: 1000
INFO:root:fifo-60: 3000
INFO:root:swf-32: 10000
INFO:root:edf-60: 2000
INFO:root:ljf-72: 0
INFO:root:fifo-44: 5000
INFO:root:edf-56: 3000
INFO:root:swf-24: 16000
INFO:root:ljf-64: 1000
INFO:root:ljf-44: 6000
INFO:root:ljf-36: 8000
INFO:

INFO:root:swf-52: 11000
INFO:root:fifo-72: 4000
INFO:root:edf-48: 9000
INFO:root:ljf-44: 11000
INFO:root:sjf-32: 12000
INFO:root:fifo-48: 9000
INFO:root:fifo-44: 11000
INFO:root:edf-24: 24000
INFO:root:ljf-24: 23000
INFO:root:edf-36: 14000
INFO:root:swf-36: 14000
INFO:root:ljf-48: 9000
INFO:root:fifo-28: 19000
INFO:root:swf-28: 20000
INFO:root:edf-32: 15000
INFO:root:fifo-24: 20000
INFO:root:ljf-32: 14000
INFO:root:sjf-44: 10000
INFO:root:swf-44: 10000
INFO:root:sjf-56: 8000
INFO:root:edf-52: 8000
INFO:root:fifo-52: 9000
INFO:root:edf-40: 11000
INFO:root:edf-28: 17000
INFO:root:edf-60: 7000
INFO:root:sjf-40: 11000
INFO:root:edf-56: 8000
INFO:root:swf-24: 24000
INFO:root:swf-32: 15000
INFO:root:sjf-48: 11000
INFO:root:sjf-36: 14000
INFO:root:ljf-52: 10000
INFO:root:fifo-68: 5000
INFO:root:ljf-32: 15000
INFO:root:edf-44: 13000
INFO:root:sjf-24: 19000
INFO:root:ljf-28: 21000
INFO:root:swf-56: 8000
INFO:root:swf-48: 9000
INFO:root:fifo-36: 14000
INFO:root:edf-48: 10000
INFO:root:fifo-40: 1

INFO:root:ljf-68: 8000
INFO:root:edf-24: 40000
INFO:root:swf-56: 12000
INFO:root:fifo-24: 27000
INFO:root:sjf-72: 8000
INFO:root:sjf-32: 18000
INFO:root:sjf-60: 12000
INFO:root:ljf-56: 12000
INFO:root:ljf-24: 36000
INFO:root:fifo-72: 8000
INFO:root:swf-24: 37000
INFO:root:edf-24: 41000
INFO:root:sjf-40: 15000
INFO:root:ljf-60: 12000
INFO:root:swf-28: 27000
INFO:root:ljf-64: 10000
INFO:root:edf-28: 26000
INFO:root:edf-36: 20000
INFO:root:ljf-40: 16000
INFO:root:ljf-24: 37000
INFO:root:ljf-72: 8000
INFO:root:fifo-64: 10000
INFO:root:edf-24: 42000
INFO:root:sjf-28: 25000
INFO:root:ljf-52: 14000
INFO:root:swf-24: 38000
INFO:root:sjf-52: 13000
INFO:root:edf-56: 13000
INFO:root:sjf-36: 19000
INFO:root:edf-24: 43000
INFO:root:fifo-56: 13000
INFO:root:ljf-24: 38000
INFO:root:swf-24: 39000
INFO:root:edf-28: 27000
INFO:root:fifo-28: 26000
INFO:root:fifo-60: 12000
INFO:root:sjf-64: 10000
INFO:root:sjf-48: 15000
INFO:root:fifo-52: 14000
INFO:root:swf-36: 20000
INFO:root:edf-24: 44000
INFO:root:swf

INFO:root:edf-44: 21000
INFO:root:edf-24: 82000
INFO:root:ljf-36: 23000
INFO:root:swf-36: 27000
INFO:root:edf-24: 83000
INFO:root:ljf-72: 11000
INFO:root:sjf-52: 16000
INFO:root:swf-24: 65000
INFO:root:fifo-44: 19000
edf-24
Cloud Cost (edf-24): 657240.5224999955
INFO:root:fifo-52: 17000
INFO:root:fifo-24: 38000
Avg Waiting Time (edf-24): 5.283784406445144e-17INFO:root:edf-28: 37000

INFO:root:edf-60: 14000
83154
INFO:root:ljf-24: 64000
INFO:root:edf-32: 25000
INFO:root:ljf-28: 40000
INFO:root:sjf-36: 23000
INFO:root:sjf-56: 15000
INFO:root:ljf-32: 26000
INFO:root:swf-24: 66000
INFO:root:swf-44: 18000
INFO:root:swf-48: 17000
INFO:root:fifo-48: 17000
INFO:root:ljf-24: 65000
INFO:root:fifo-24: 39000
INFO:root:edf-56: 16000
INFO:root:swf-24: 67000
INFO:root:edf-48: 19000
INFO:root:ljf-28: 41000
INFO:root:fifo-56: 16000
INFO:root:ljf-24: 66000
INFO:root:swf-56: 15000
INFO:root:ljf-44: 19000
INFO:root:swf-24: 68000
INFO:root:swf-32: 25000
INFO:root:fifo-28: 31000
INFO:root:ljf-64: 13000
INFO

...

INFO:root:sjf-44: 66000
INFO:root:ljf-52: 68000
INFO:root:sjf-60: 42000
INFO:root:fifo-52: 51000
INFO:root:lvjf-40: 3000
INFO:root:fifo-48: 67000
INFO:root:sjf-48: 56000
INFO:root:edf-56: 53000
INFO:root:lwf-36: 14000
INFO:root:ljf-60: 48000
INFO:root:ljf-48: 61000
INFO:root:ljf-64: 32000
INFO:root:svjf-40: 4000
INFO:root:ljf-52: 69000
INFO:root:sjf-44: 67000
INFO:root:ldf-32: 10000
INFO:root:lifo-24: 17000
INFO:root:ljf-56: 38000
INFO:root:fifo-48: 68000
INFO:root:lvjf-36: 8000
INFO:root:sjf-52: 56000
INFO:root:fifo-56: 46000
INFO:root:ljf-52: 70000
INFO:root:svjf-24: 15000
INFO:root:sjf-44: 68000
INFO:root:fifo-52: 52000
INFO:root:sjf-48: 57000
INFO:root:lvjf-28: 21000
INFO:root:edf-56: 54000
INFO:root:ljf-48: 62000
INFO:root:ljf-52: 71000
INFO:root:sjf-72: 31000
INFO:root:fifo-48: 69000
INFO:root:edf-68: 33000
INFO:root:fifo-60: 37000
INFO:root:sjf-44: 69000
INFO:root:svjf-36: 10000
INFO:root:lwf-44: 0
INFO:root:ldf-44: 4000
INFO:root:swf-56: 39000
INFO:root:ljf-52: 72000
INFO:root:

INFO:root:sjf-48: 80000
INFO:root:sjf-52: 75000
INFO:root:fifo-56: 57000
INFO:root:ljf-64: 34000
INFO:root:edf-52: 46000
INFO:root:sjf-56: 40000
INFO:root:lwf-24: 25000
INFO:root:sjf-60: 51000
INFO:root:swf-60: 39000
INFO:root:lifo-36: 13000
INFO:root:sjf-48: 81000
INFO:root:fifo-52: 66000
INFO:root:lwf-28: 18000
INFO:root:sjf-48: 82000
INFO:root:ljf-60: 60000
INFO:root:sjf-52: 76000
INFO:root:edf-56: 69000
INFO:root:sjf-48: 83000
INFO:root:edf-60: 45000
INFO:root:lifo-28: 17000
INFO:root:ldf-36: 13000
INFO:root:ldf-32: 12000
sjf-48
Cloud Cost (sjf-48): 402093.40861111204
Avg Waiting Time (sjf-48): 4.5067572878502695e-17
83154
INFO:root:sjf-52: 77000
INFO:root:lifo-32: 11000
INFO:root:swf-56: 50000
INFO:root:edf-56: 70000
INFO:root:fifo-56: 58000
INFO:root:fifo-52: 67000
INFO:root:edf-52: 47000
INFO:root:ljf-60: 61000
INFO:root:ljf-56: 44000
INFO:root:sjf-60: 52000
INFO:root:lwf-48: 3000
INFO:root:sjf-52: 78000
INFO:root:swf-68: 30000
INFO:root:edf-64: 33000
INFO:root:edf-56: 71000
INF

INFO:root:sjf-60: 68000
INFO:root:lvjf-28: 27000
INFO:root:swf-56: 65000
INFO:root:ldf-36: 15000
INFO:root:edf-52: 59000
INFO:root:lifo-48: 3000
INFO:root:edf-60: 55000
INFO:root:fifo-72: 36000
INFO:root:ldf-24: 36000
INFO:root:ldf-28: 26000
INFO:root:ldf-32: 14000
INFO:root:sjf-60: 69000
INFO:root:edf-68: 43000
INFO:root:ljf-72: 35000
INFO:root:svjf-44: 3000
INFO:root:swf-56: 66000
INFO:root:svjf-28: 18000
INFO:root:swf-64: 38000
INFO:root:lvjf-40: 7000
INFO:root:sjf-56: 46000
INFO:root:ljf-56: 56000
INFO:root:swf-60: 46000
INFO:root:edf-64: 35000
INFO:root:sjf-60: 70000
INFO:root:lwf-28: 21000
INFO:root:svjf-36: 14000
INFO:root:edf-52: 60000
INFO:root:ldf-24: 37000
INFO:root:lifo-40: 11000
INFO:root:swf-56: 67000
INFO:root:lifo-32: 13000
INFO:root:sjf-60: 71000
INFO:root:edf-68: 44000
INFO:root:ldf-56: 1000
INFO:root:edf-60: 56000
INFO:root:edf-52: 61000
INFO:root:swf-56: 68000
INFO:root:sjf-60: 72000
INFO:root:lifo-28: 20000
INFO:root:fifo-60: 46000
INFO:root:ljf-56: 57000
INFO:root

INFO:root:edf-68: 54000
INFO:root:fifo-68: 36000
INFO:root:sjf-56: 60000
INFO:root:lvjf-28: 33000
INFO:root:ljf-56: 81000
INFO:root:fifo-60: 57000
INFO:root:ljf-72: 39000
INFO:root:ljf-56: 82000
INFO:root:ljf-64: 40000
INFO:root:edf-60: 76000
INFO:root:swf-60: 60000
INFO:root:edf-64: 38000
INFO:root:ldf-24: 54000
INFO:root:ljf-56: 83000
INFO:root:lwf-32: 16000
INFO:root:sjf-56: 61000
INFO:root:sjf-72: 40000
INFO:root:lifo-52: 3000
ljf-56
Cloud Cost (ljf-56): 322771.22500000143INFO:root:lwf-28: 24000

Avg Waiting Time (ljf-56): 2.1756759320656475e-17
83154
INFO:root:edf-60: 77000
INFO:root:sjf-64: 42000
INFO:root:lvjf-48: 5000
INFO:root:lifo-24: 26000
INFO:root:swf-60: 61000
INFO:root:swf-64: 42000
INFO:root:lvjf-28: 34000
INFO:root:lwf-36: 22000
INFO:root:fifo-60: 58000
INFO:root:edf-68: 55000
INFO:root:lvjf-44: 10000
INFO:root:edf-60: 78000
INFO:root:ldf-24: 55000
INFO:root:lwf-24: 32000
INFO:root:sjf-56: 62000
INFO:root:ldf-36: 18000
INFO:root:swf-60: 62000
INFO:root:fifo-64: 43000
I

INFO:root:fifo-64: 51000
INFO:root:sjf-72: 48000
INFO:root:lvjf-48: 7000
INFO:root:lifo-24: 29000
INFO:root:ljf-68: 40000
INFO:root:ljf-64: 47000
INFO:root:edf-64: 42000
INFO:root:fifo-72: 49000
INFO:root:lvjf-24: 34000
INFO:root:fifo-60: 81000
INFO:root:lifo-28: 26000
INFO:root:sjf-68: 40000
INFO:root:edf-68: 71000
INFO:root:lwf-24: 40000
INFO:root:fifo-60: 82000
INFO:root:lwf-28: 27000
INFO:root:swf-68: 37000
INFO:root:svjf-60: 3000
INFO:root:fifo-60: 83000
INFO:root:lvjf-28: 46000
fifo-60
Cloud Cost (fifo-60): 290976.7825000008INFO:root:lifo-52: 5000

INFO:root:edf-68: 72000
Avg Waiting Time (fifo-60): 3.2635138980984715e-17
INFO:root:lwf-48: 9000
83154
INFO:root:fifo-68: 40000
INFO:root:swf-64: 50000
INFO:root:sjf-64: 48000
INFO:root:ljf-64: 48000
INFO:root:svjf-48: 6000
INFO:root:fifo-64: 52000
INFO:root:edf-68: 73000
INFO:root:sjf-72: 49000
INFO:root:lvjf-28: 47000
INFO:root:fifo-72: 50000
INFO:root:ljf-72: 45000
INFO:root:lifo-32: 18000
INFO:root:svjf-24: 27000
INFO:root:lwf-24: